<a href="https://colab.research.google.com/github/dp-93/Random-Forest-Model-MNIST-/blob/main/MNIST_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

In [6]:
import tensorflow as tf
from tensorflow import keras
import gradio as gr
import numpy as np
from PIL import Image

# --- 1. Load Data and Train a Model (Same as before) ---
(X_train, y_train), (_, _) = keras.datasets.mnist.load_data()
X_train = X_train / 255.0
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Training the model...")
model.fit(X_train, y_train, epochs=5)
print("Model trained successfully!")


# --- 2. Define the FINAL Prediction Function ---
def predict_digit(sketchpad_dict):
    image = sketchpad_dict['composite']
    pil_image = Image.fromarray(image).convert('L')
    resized_image = pil_image.resize((28, 28))
    img_array = np.array(resized_image)
    img_array = 255 - img_array

    # --- NEW STEP: Increase Contrast ---
    # This makes the input look more like the training data.
    # We set any pixel with a value less than 50 to 0 (black).
    threshold = 50
    img_array[img_array < threshold] = 0

    img_array = img_array / 255.0
    img_array = img_array.reshape(1, 28, 28)

    prediction_probs = model.predict(img_array)[0]
    confidence_scores = {str(i): float(prob) for i, prob in enumerate(prediction_probs)}
    return confidence_scores


# --- 3. Create and Launch the Gradio Interface ---
interface = gr.Interface(fn=predict_digit,
                         inputs=gr.Sketchpad(type="numpy"),
                         outputs=gr.Label(num_top_classes=3),
                         title="MNIST Digit Recognizer",
                         description="Draw a digit (0-9) and see the model's prediction.")

interface.launch()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training the model...
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8802 - loss: 0.4327
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9642 - loss: 0.1243
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9759 - loss: 0.0795
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9833 - loss: 0.0575
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9876 - loss: 0.0416
Model trained successfully!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2ac9e6ca5ff3e4ff4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy t